In [67]:
import numpy as np
from itertools import combinations

In [68]:
dim = int(input('Inserire numero di variabili della funzione obiettivo: '))

if type(dim)!=int:
    raise ValueError('DEVE ESSERE INTERO, BRUTTA CACCA')

coeff = input('Inserire coefficienti della funzione obiettivo, separandoli con lo spazio: ')
list_coff = coeff.split()

if len(list_coff)!=dim:
    raise ValueError('Dimensione sbagliata.')

list_coff = [int(i) for i in list_coff]
min = input('Digitare \'min\' per un problema di minimizzazione, altrimenti \'max\' per un problema di massimizzazione: ')

if min=='min':
    pass
elif min=='max':
    list_coff = [-i for i in list_coff]
else:
    raise ValueError('Specificare \'min\' o \'max\'')

n_vincoli = int(input('Inserire numero di vincoli: ')) #righe
n_variabili = int(input('Inserire numero totale di variabili dei vincoli: '))
vincoli = np.zeros((n_vincoli, n_variabili))
termini_noti = np.zeros((n_vincoli, 1))

In [69]:
tipi_vincolo = []

for v, k in zip(vincoli, range(vincoli.shape[0])):
    print('\nVincolo numero', k)
    tipo = input('Inserire tipo di vincolo (\'<=\', \'>=\', \'=\'): ')
    if tipo!='<=' and tipo!='>=' and tipo!='=':
        raise ValueError('Tipo invalido.')
    tipi_vincolo.append(tipo)

    for j in range(v.shape[0]):
        print('Inserire coefficiente numero', j)
        v[j] = int(input(''))   
    print('Inserire termine noto: ')
    termini_noti[k] = int(input(''))


Vincolo numero 0
Inserire coefficiente numero 0
Inserire coefficiente numero 1
Inserire coefficiente numero 2
Inserire termine noto: 

Vincolo numero 1
Inserire coefficiente numero 0
Inserire coefficiente numero 1
Inserire coefficiente numero 2
Inserire termine noto: 


In [70]:
print(vincoli)
print(termini_noti)

[[ 3. -1.  1.]
 [ 2.  1.  2.]]
[[ 4.]
 [10.]]


Ogni problema di programmazione lineare può essere trasformato in un problema equivalente formulato in forma standard.

Definizione: Un problema di PL si dice formulato in forma standard se si tratta di minimizzare una funzione obiettivo lineare soggetta a vincoli di uguaglianza, se le variabili di decisione sono vincolate ad essere nonnegative e il vettore dei termini noti è nonnegativo.

Infatti, è sempre possibile assumere che la funzione obiettivo lineare z(x) sia da
minimizzare, in quanto il valore della variabile x∗ che rende massima z(x)
rende minima la stessa funzione cambiata di segno.

Un qualsiasi vincolo può essere trasformato in vincolo di uguaglianza introducendo opportune variabili aggiuntive nonnegative: 
1. nel caso di un vincolo (<=) avremo una variabile di slack (positiva, che rappresenta la differenza nonnegativa tra il secondo ed il primo
membro della disuguaglianza);
2. nel caso di un vincolo (>=) avremo una variabile di surplus (negativa, rappresenta la differenza nonnegativa tra il primo ed il secondo
membro della disuguaglianza).

In [71]:
##### standardizziamo il problema #####
# termini noti negativi no bueno (cambio il verso della disequazione)
for w in range(vincoli.shape[0]):
    if vincoli[w][vincoli.shape[1]-1]<0:
        vincoli[w] = [-i for i in vincoli[w]]
        if tipi_vincolo[w]=='<=':
            tipi_vincolo[w]='>='
        elif tipi_vincolo[w]=='>=':
            tipi_vincolo[w]='<='

# dobbiamo aggiungere un numero di variabili di surplus e slack pari al numero di disequazioni
num_diseq = tipi_vincolo.count('<=') + tipi_vincolo.count('>=')
# aggiungo tanti zeri quante sono le variabili di surplus/slack alla funzione obiettivo
list_coff = list_coff + num_diseq*[0]
# creo il vettore di zeri da sovrascrivere in seguito
surplack_col = np.zeros((len(tipi_vincolo), num_diseq))

for tipo, i in zip(tipi_vincolo, range(surplack_col.shape[1])):
    if tipo=='<=':
    # variabile di slack
        surplack_col[i][i] = 1
    if tipo=='>=':
    # variabile di surplus
        surplack_col[i][i] = -1

matrix_A = np.concatenate((vincoli, surplack_col), axis=1)
matrix_Ab = np.concatenate((matrix_A, termini_noti), axis=1)
rank_A = np.linalg.matrix_rank(matrix_A)
rank_Ab = np.linalg.matrix_rank(matrix_Ab)
if rank_A != rank_Ab:
    raise ValueError('Rango non ammissibile: rank(A) != rank(Ab)')

if rank_A != len(tipi_vincolo):
    raise ValueError('Rango non ammissibile: rank_(A) != numero vincoli, rank(A):', rank_A)

In [72]:
print(matrix_A)
print(termini_noti)

[[ 3. -1.  1.  1.  0.]
 [ 2.  1.  2.  0. -1.]]
[[ 4.]
 [10.]]


Se un problema di PL formulato in forma standard ammette una soluzione ottima finita, allora
esiste una soluzione ammissibile di base ottima.

Il metodo del simplesso si basa su tale considerazione e cerca una soluzione ottima al problema
muovendosi da una soluzione ammissibile di base ad una adiacente sempre nella direzione lungo la quale si verifica un decremento del valore corrente della funzione obiettivo.

Il metodo del simplesso, passando da una soluzione ammissibile di base ad una soluzione ammissibile di base adiacente e terminando quando non esiste una soluzione ammissibile di base
migliore, può essere interpretato come algoritmo di ricerca locale. Fortunatamente, però, nel caso di problemi di PL, dal momento che si tratta di ottimizzare una funzione obiettivo convessa su una regione ammissibile convessa, si è certi che l’ottimo locale restituito in output dal metodo del simplesso è ottimo anche globalmente.